# Testing your TCLab

## TCLab Objects

### 1. Testing the TCLab connection

The following cell tests the serial connection to the TCLab hardware. The output of this cell will report:

* version number of the tclab Python library
* device port and speed of the serial connection
* version number of the TCLab firmware

In [13]:
from tclab import TCLab

with TCLab() as lab:
    pass

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem142101 at 115200 baud.
TCLab Firmware 1.5.1 Arduino Leonardo/Micro.
TCLab disconnected successfully.


### 2. Flash the TCLab LED

The `lab.LED()` function provides a convenient tool for testing the connection to the TCLab hardware. To set the LED to Y% of maximum, use

    lab.LED(Y)
    
The following code flashes the LED to 100% five times.

In [26]:
from tclab import TCLab
import time

tperiod = 0.5
with TCLab() as lab:
    for k in range(0, 5):        # repeat 5 times
        lab.LED(100)             # LED set to 100%
        time.sleep(tperiod/2)    # sleep for 50% of cycle time
        lab.LED(0)               # LED set to 0%
        time.sleep(tperiod/2)    # sleep for 50% of cycle time

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem142101 at 115200 baud.
TCLab Firmware 1.5.1 Arduino Leonardo/Micro.
TCLab disconnected successfully.


### 3. Read the temperature sensors

The temperatures measured by the thermisters located on the TCLab are accessed with `lab.T1` and `lab.T2` where `lab` is a `TCLab` instance. The following cell will access and print the temperatures of the two thermisters.

In [6]:
from tclab import TCLab

with TCLab() as lab:
    print("Temperature 1: {0:0.2f} C".format(lab.T1))
    print("Temperature 2: {0:0.2f} C".format(lab.T2))

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem142101 at 115200 baud.
TCLab Firmware 1.5.1 Arduino Leonardo/Micro.
Temperature 1: 23.16 C
Temperature 2: 22.52 C
TCLab disconnected successfully.


### 4. Turn the heaters on and off

In [17]:
from tclab import TCLab
import time

with TCLab() as lab:
    print()
    print("Starting Temperature 1: {0:0.2f} C".format(lab.T1), flush=True)
    print("Starting Temperature 2: {0:0.2f} C".format(lab.T2), flush=True)
    
    print()
    print("Set Heater 1:", lab.Q1(100), "%", flush=True)
    print("Set Heater 2:", lab.Q2(100), "%", flush=True)
    
    t_heat = 30
    print()
    print("Heat for", t_heat, "seconds")
    time.sleep(t_heat)

    print()
    print("Set Heater 1:", lab.Q1(0), "%",flush=True)
    print("Set Heater 2:", lab.Q2(0), "%",flush=True)
    
    print()
    print("Final Temperature 1: {0:0.2f} C".format(lab.T1))
    print("Final Temperature 2: {0:0.2f} C".format(lab.T2))
    print()

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem142101 at 115200 baud.
TCLab Firmware 1.5.1 Arduino Leonardo/Micro.

Starting Temperature 1: 20.26 C
Starting Temperature 2: 20.26 C

Set Heater 1: 100.0 %
Set Heater 2: 100.0 %

Heat for 30 seconds

Set Heater 1: 0.0 %
Set Heater 2: 0.0 %

Final Temperature 1: 20.26 C
Final Temperature 2: 20.26 C

TCLab disconnected successfully.


## Synchronizing operation with the wall clock using `clock`

### 1. Use `clock` iterator

In [12]:
from tclab import TCLab, clock

tperiod = 20

# connect to the temperature control lab
with TCLab() as lab:
    # turn heaters on
    print()
    print("Set Heater 1 to {0:0.1f} %".format(lab.Q1(100)))
    print("Set Heater 2 to {0:0.1f} %".format(lab.Q2(100)))

    # report temperatures for the next tperiod seconds
    print()
    for t in clock(tperiod):
        print("   {0:5.1f} sec:   T1 = {1:0.1f} C    T2 = {2:0.1f} C".format(t, lab.T1, lab.T2), flush=True)

    print()

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem142101 at 115200 baud.
TCLab Firmware 1.5.1 Arduino Leonardo/Micro.

Set Heater 1 to 100.0 %
Set Heater 2 to 100.0 %

     0.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     1.0 sec:   T1 = 22.8 C    T2 = 22.2 C
     2.0 sec:   T1 = 23.2 C    T2 = 22.5 C
     3.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     4.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     5.0 sec:   T1 = 23.2 C    T2 = 22.5 C
     6.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     7.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     8.0 sec:   T1 = 22.8 C    T2 = 22.5 C
     9.0 sec:   T1 = 23.2 C    T2 = 22.5 C
    10.0 sec:   T1 = 22.8 C    T2 = 22.5 C
    11.0 sec:   T1 = 23.2 C    T2 = 22.5 C
    12.0 sec:   T1 = 22.8 C    T2 = 22.5 C
    13.0 sec:   T1 = 23.2 C    T2 = 22.5 C
    14.0 sec:   T1 = 22.8 C    T2 = 22.5 C
    15.0 sec:   T1 = 22.8 C    T2 = 22.5 C
    16.0 sec:   T1 = 22.8 C    T2 = 22.5 C
    17.0 sec:   T1 = 23.2 C    T2 = 22.5 C
    18.0 sec:   T1 = 22.8 C    T2 = 2